### Download, tokenize, and split datasets


1. Wikitext - `load_dataset("iohadrubin/wikitext-103-raw-v1")`
2. Fineweb - `load_dataset("HuggingFaceFW/fineweb-edu", name="CC-MAIN-2024-10", split="train", streaming=True)`
3. Slimpajama `load_dataset("cerebras/SlimPajama-627B")`

In [13]:
from datasets import load_dataset
from pathlib import Path
from tqdm import tqdm
import tiktoken
import numpy as np
import os

In [14]:
raw_path = Path('/Users/djemec/data/gpt_train_data/raw')
proc_path = Path('/Users/djemec/data/gpt_train_data/processed')
shard_size = int(1e8)

### Setup tokenizer

In [15]:
# init the tokenizer
enc = tiktoken.get_encoding('gpt2')
eot = enc._special_tokens['<|endoftext|>'] # end of text token

def tokenize(doc):
    # tokenizes a single document and returns a numpy array of uint16 tokens
    tokens = [eot] # the special <|endoftext|> token delimits all documents
    tokens.extend(enc.encode_ordinary(doc['text']))
    tokens_np = np.array(tokens)
    assert (0 <= tokens_np).all() and (tokens_np < 2**16).all(), "token dictionary too large for uint16"
    tokens_np_uint16 = tokens_np.astype(np.uint16)
    return tokens_np_uint16

In [16]:
def write_datafile(filename, tokens_np):
    np.save(filename, tokens_np)

#### Wikitext

In [7]:
wt_path = raw_path / 'wikitext103/'
wt_path.mkdir(parents=True, exist_ok=True)

In [11]:
wt_tr = load_dataset('iohadrubin/wikitext-103-raw-v1', split='train')

In [21]:
i = 0
for example in wt_tr:
    print(example['text'])
    i += 1
    if i > 2:
        break
    

= Valkyria Chronicles III =
Senjō no Valkyria 3: Unrecorded Chronicles (Japanese: 戦場のヴァルキュリア3, lit. Valkyria of the Battlefield 3), commonly referred to as Valkyria Chronicles III outside Japan, is a tactical role-playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan, it is the third game in the Valkyria series. Employing the same fusion of tactical and real-time gameplay as its predecessors, the story runs parallel to the first game and follows the "Nameless", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit "Calamaty Raven".
The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Hon